<a href="https://colab.research.google.com/github/tsabbir96/spam-article-checker/blob/main/Fake_news_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import necessary packages
from google.colab import drive
drive.mount("/content/gdrive")
 
import pandas as pd
import os
import re
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
import pandas as pd
import numpy as np
 
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Mounted at /content/gdrive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#get data
data_real = pd.read_csv('./gdrive/My Drive/real_news_set.csv')
data_real.head()

,url,headline,body,label
0,https://web.archive.org/web/20161003021917/htt...,Rep. Kristi Noem: My father’s tragic death and...,"By the time I got to the farm, neighbors and f...",1
1,https://web.archive.org/web/20161004070028/htt...,Russia deploys advanced anti-missile system to...,It comes after Russia’s actions led to the col...,1
2,https://web.archive.org/web/20161004070028/htt...,"Aleppo: ""The international community has turne...","Aleppo has become an abandoned city, reduced t...",1
3,https://web.archive.org/web/20161005034102/htt...,"Pence, Kaine trade attacks, talk over each oth...","Within 10 minutes, they were talking over each...",1
4,https://web.archive.org/web/20161006061117/htt...,US taxpayers made millionaires out of Afghan g...,"A total of $114 billion, which does not includ...",1


In [3]:
## Shape
print("shape: ", data_real.shape)

shape:  (11831, 4)


In [4]:
## Checking for Missing values
data_real.isnull().sum()

url         0
headline    0
body        0
label       0
dtype: int64

In [5]:
data_fake = pd.read_csv('./gdrive/My Drive/fake_news_set.csv')
data_fake.head()
## Shape
print("shape: ", data_fake.shape)
data_fake.isnull().sum()
data = pd.concat([data_real,data_fake]) #concatenate bothe the dataset into one dataset
# data = data.sample(frac=1).reset_index(drop=True)#shuffle the data
data = data.sample(frac=1).reset_index(drop=True)
data


shape:  (6822, 4)


,url,headline,body,label
0,https://web.archive.org/web/20161029002704/htt...,Hillary Clinton’s campaign sounds pretty angry...,This post has been updated with Hillary C...,1
1,http://www.reuters.com/article/us-usa-trump-ut...,Trump’s Carrier deal worthy of ’banana’ republ...,U. S. Donald Trump’s intervention to stop j...,1
2,https://www.reuters.com/article/us-totalerg-m-...,Exclusive: Total & Erg aim to wrap up Italian ...,FILE PHOTO: A customer holds a gas pump as he ...,1
3,kingworldnews.com,Today’s Upside Move In Gold & Silver Is Nothin...,"29 Views November 01, 2016 GOLD , KWN King Wor...",0
4,http://www.npr.org/2016/10/21/498736729/steine...,"Steinem: For Feminism, A Clinton Win Would Be ...","Make no mistake. Gloria Steinem, noted feminis...",1
...,...,...,...,...
18648,https://www.nytimes.com/2017/10/06/theater/dav...,"A Eugene O’Neill Marathon: 1 Actor, 1 Script, ...",Mr. Greenspan is also no stranger to lengthy e...,1
18649,http://www.nationalreview.com/article/443223/e...,I Was a Florida Elector: The People Who Begged...,"It was supposed to have ended on November 8, 2...",1
18650,http://www.reuters.com/article/us-usa-thanksgi...,"Thanksgiving cheer for consumers as turkey, ha...",U. S. consumers can expect to spend less on t...,1
18651,https://www.reuters.com/article/us-abertis-m-a...,Spanish regulator clears the way for Atlantia'...,FILE PHOTO: The Abertis's logo is seen during ...,1


In [6]:
#get length of the body of the article
length = []
[length.append(len(str(text))) for text in data['body']]
data['length'] = length#store the length in different coloumn
data.head()

,url,headline,body,label,length
0,https://web.archive.org/web/20161029002704/htt...,Hillary Clinton’s campaign sounds pretty angry...,This post has been updated with Hillary C...,1,5258
1,http://www.reuters.com/article/us-usa-trump-ut...,Trump’s Carrier deal worthy of ’banana’ republ...,U. S. Donald Trump’s intervention to stop j...,1,2917
2,https://www.reuters.com/article/us-totalerg-m-...,Exclusive: Total & Erg aim to wrap up Italian ...,FILE PHOTO: A customer holds a gas pump as he ...,1,2921
3,kingworldnews.com,Today’s Upside Move In Gold & Silver Is Nothin...,"29 Views November 01, 2016 GOLD , KWN King Wor...",0,3121
4,http://www.npr.org/2016/10/21/498736729/steine...,"Steinem: For Feminism, A Clinton Win Would Be ...","Make no mistake. Gloria Steinem, noted feminis...",1,2509


In [7]:
## Minimum Length
print("minimum length of texts: ", min(data['length']))
## Maximum Length
print("maximum length of texts: ", max(data['length']))

minimum length of texts:  1
maximum length of texts:  142961


In [8]:
## Checking for outliers
## typically I think a news with a text length of less than 100 characters is not a news at all
## I think its a glitch
## Checking how many rows have less than 100 characters

print("Number of Outliers: ", len(data[data['length'] < 100]))

## Lets have a look at those texts that I believe they are outliers
data[['body','length']][data['length'] < 100]

Number of Outliers:  193


,body,length
347,Chat with us in Facebook Messenger. Find out w...,89
356,Chat with us in Facebook Messenger. Find out w...,89
387,Chat with us in Facebook Messenger. Find out w...,89
389,Chat with us in Facebook Messenger. Find out w...,89
605,Chat with us in Facebook Messenger. Find out w...,89
...,...,...
18091,"Vietnam Is in Great Danger, You Must Publish a...",83
18235,About the EU and NATO . (Thanks to Daniel McAd...,79
18330,"In the Weinstein Scandal, it Is Hollywood’s Co...",92
18446,"Vietnam Is in Great Danger, You Must Publish a...",83


In [9]:
# dropping the outliers
data = data.drop(data['body'][data['length'] < 100].index, axis = 0)
data.shape

(18460, 5)

In [10]:
# dropping ALL duplicte values 
data.drop_duplicates(subset ="body", 
                     keep = False, inplace = True)
data.shape

(16925, 5)

In [11]:
# #dropping rows with empty "labels"
# data['label'].dropna(inplace=True)
# print(data['label'].unique())
# print(data.shape)

In [12]:
data = data.reset_index()
data

,index,url,headline,body,label,length
0,0,https://web.archive.org/web/20161029002704/htt...,Hillary Clinton’s campaign sounds pretty angry...,This post has been updated with Hillary C...,1,5258
1,1,http://www.reuters.com/article/us-usa-trump-ut...,Trump’s Carrier deal worthy of ’banana’ republ...,U. S. Donald Trump’s intervention to stop j...,1,2917
2,2,https://www.reuters.com/article/us-totalerg-m-...,Exclusive: Total & Erg aim to wrap up Italian ...,FILE PHOTO: A customer holds a gas pump as he ...,1,2921
3,3,kingworldnews.com,Today’s Upside Move In Gold & Silver Is Nothin...,"29 Views November 01, 2016 GOLD , KWN King Wor...",0,3121
4,4,http://www.npr.org/2016/10/21/498736729/steine...,"Steinem: For Feminism, A Clinton Win Would Be ...","Make no mistake. Gloria Steinem, noted feminis...",1,2509
...,...,...,...,...,...,...
16920,18648,https://www.nytimes.com/2017/10/06/theater/dav...,"A Eugene O’Neill Marathon: 1 Actor, 1 Script, ...",Mr. Greenspan is also no stranger to lengthy e...,1,4993
16921,18649,http://www.nationalreview.com/article/443223/e...,I Was a Florida Elector: The People Who Begged...,"It was supposed to have ended on November 8, 2...",1,2999
16922,18650,http://www.reuters.com/article/us-usa-thanksgi...,"Thanksgiving cheer for consumers as turkey, ha...",U. S. consumers can expect to spend less on t...,1,2834
16923,18651,https://www.reuters.com/article/us-abertis-m-a...,Spanish regulator clears the way for Atlantia'...,FILE PHOTO: The Abertis's logo is seen during ...,1,2614


In [13]:
## Minimum Length
print("minimum length of texts: ", min(data['length']))
## Maximum Length
print("maximum length of texts: ", max(data['length']))

max_features = 142900

minimum length of texts:  101
maximum length of texts:  142961


In [14]:
# Change all the text to lower case.
data['body'] = [entry.lower() for entry in data['body']]
data.head()

,index,url,headline,body,label,length
0,0,https://web.archive.org/web/20161029002704/htt...,Hillary Clinton’s campaign sounds pretty angry...,this post has been updated with hillary c...,1,5258
1,1,http://www.reuters.com/article/us-usa-trump-ut...,Trump’s Carrier deal worthy of ’banana’ republ...,u. s. donald trump’s intervention to stop j...,1,2917
2,2,https://www.reuters.com/article/us-totalerg-m-...,Exclusive: Total & Erg aim to wrap up Italian ...,file photo: a customer holds a gas pump as he ...,1,2921
3,3,kingworldnews.com,Today’s Upside Move In Gold & Silver Is Nothin...,"29 views november 01, 2016 gold , kwn king wor...",0,3121
4,4,http://www.npr.org/2016/10/21/498736729/steine...,"Steinem: For Feminism, A Clinton Win Would Be ...","make no mistake. gloria steinem, noted feminis...",1,2509


In [15]:
#Tokenization : In this each entry in the corpus will be broken into set of words
data['body']= [word_tokenize(entry) for entry in data['body']]

In [16]:
data

,index,url,headline,body,label,length
0,0,https://web.archive.org/web/20161029002704/htt...,Hillary Clinton’s campaign sounds pretty angry...,"[this, post, has, been, updated, with, hillary...",1,5258
1,1,http://www.reuters.com/article/us-usa-trump-ut...,Trump’s Carrier deal worthy of ’banana’ republ...,"[u., s., donald, trump, ’, s, intervention, to...",1,2917
2,2,https://www.reuters.com/article/us-totalerg-m-...,Exclusive: Total & Erg aim to wrap up Italian ...,"[file, photo, :, a, customer, holds, a, gas, p...",1,2921
3,3,kingworldnews.com,Today’s Upside Move In Gold & Silver Is Nothin...,"[29, views, november, 01, ,, 2016, gold, ,, kw...",0,3121
4,4,http://www.npr.org/2016/10/21/498736729/steine...,"Steinem: For Feminism, A Clinton Win Would Be ...","[make, no, mistake, ., gloria, steinem, ,, not...",1,2509
...,...,...,...,...,...,...
16920,18648,https://www.nytimes.com/2017/10/06/theater/dav...,"A Eugene O’Neill Marathon: 1 Actor, 1 Script, ...","[mr., greenspan, is, also, no, stranger, to, l...",1,4993
16921,18649,http://www.nationalreview.com/article/443223/e...,I Was a Florida Elector: The People Who Begged...,"[it, was, supposed, to, have, ended, on, novem...",1,2999
16922,18650,http://www.reuters.com/article/us-usa-thanksgi...,"Thanksgiving cheer for consumers as turkey, ha...","[u., s., consumers, can, expect, to, spend, le...",1,2834
16923,18651,https://www.reuters.com/article/us-abertis-m-a...,Spanish regulator clears the way for Atlantia'...,"[file, photo, :, the, abertis, 's, logo, is, s...",1,2614


In [17]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
i=0
for index,entry in enumerate(data['body']):

    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    data.loc[index,'text_final'] = str(Final_words)
    print(index)


Streaming output truncated to the last 5000 lines.
11925
11926
11927
11928
11929
11930
11931
11932
11933
11934
11935
11936
11937
11938
11939
11940
11941
11942
11943
11944
11945
11946
11947
11948
11949
11950
11951
11952
11953
11954
11955
11956
11957
11958
11959
11960
11961
11962
11963
11964
11965
11966
11967
11968
11969
11970
11971
11972
11973
11974
11975
11976
11977
11978
11979
11980
11981
11982
11983
11984
11985
11986
11987
11988
11989
11990
11991
11992
11993
11994
11995
11996
11997
11998
11999
12000
12001
12002
12003
12004
12005
12006
12007
12008
12009
12010
12011
12012
12013
12014
12015
12016
12017
12018
12019
12020
12021
12022
12023
12024
12025
12026
12027
12028
12029
12030
12031
12032
12033
12034
12035
12036
12037
12038
12039
12040
12041
12042
12043
12044
12045
12046
12047
12048
12049
12050
12051
12052
12053
12054
12055
12056
12057
12058
12059
12060
12061
12062
12063
12064
12065
12066
12067
12068
12069
12070
12071
12072
12073
12074
12075
12076
12077
12078
12079
12080
12081
12082
1

In [18]:
#split data into train test
Train_X, Test_X, Train_Y, Test_Y = train_test_split(data['text_final'],data['label'],test_size=0.3)

In [19]:
Train_Y

10133    1
13795    0
3615     1
12741    1
9327     1
        ..
10778    1
14352    1
11743    1
14020    1
14016    0
Name: label, Length: 11847, dtype: int64

In [20]:
data.dtypes

index          int64
url           object
headline      object
body          object
label          int64
length         int64
text_final    object
dtype: object

In [21]:
#create the tokenizer
Tfidf_vect = TfidfVectorizer(max_features=7000)
#fit the tokenizer
Tfidf_vect.fit(data['text_final'])

#saving the tokenizer so that we ccan use it later to preprocess the text before prediction
filename = 'tokenizer_news.sav'
pickle.dump(Tfidf_vect, open(filename, 'wb'))

#transform the data into vectors using the tokenizer
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [22]:

from google.colab import files
files.download('tokenizer_news.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
print(Tfidf_vect.vocabulary_)

{'post': 4787, 'update': 6635, 'hillary': 2940, 'clinton': 1118, 'comment': 1200, 'campaign': 871, 'could': 1421, 'go': 2712, 'different': 1758, 'way': 6822, 'response': 5297, 'bombshell': 709, 'news': 4225, 'fbi': 2382, 'look': 3733, 'email': 2055, 'relate': 5199, 'investigation': 3314, 'democratic': 1654, 'nominee': 4260, 'angry': 265, 'statement': 5955, 'chairman': 991, 'john': 3413, 'podesta': 4730, 'remarkable': 5222, 'begin': 576, 'suggest': 6088, 'least': 3597, 'partially': 4535, 'political': 4748, 'republicans': 5262, 'continue': 1351, 'handle': 2836, 'along': 211, 'director': 1782, 'james': 3373, 'comey': 1193, 'july': 3443, 'conclusion': 1266, 'reasonable': 5115, 'prosecutor': 4948, 'would': 6938, 'bring': 783, 'charge': 1011, 'upon': 6640, 'complete': 1237, 'three': 6335, 'month': 4076, 'ago': 156, 'declare': 1598, 'move': 4116, 'forward': 2546, 'case': 927, 'like': 3669, 'add': 83, 'even': 2208, 'close': 1121, 'call': 863, 'say': 5506, 'since': 5726, 'donald': 1897, 'trump'

In [24]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  84.56085072863331


In [25]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  93.04844426939741


In [26]:
## training - LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_LR = logisticRegr.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("LR Accuracy Score -> ",accuracy_score(predictions_LR, Test_Y)*100)

LR Accuracy Score ->  91.80779834580544


In [27]:

## training - RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_RFC = rfc.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("RFC Accuracy Score -> ",accuracy_score(predictions_RFC, Test_Y)*100)

RFC Accuracy Score ->  87.33753446238677


In [28]:
filename = 'naive_news.sav'
pickle.dump(Naive, open(filename, 'wb'))

In [29]:
filename = 'svm_news.sav'
pickle.dump(SVM, open(filename, 'wb'))

In [30]:
filename = 'LogReg_news.sav'
pickle.dump(logisticRegr, open(filename, 'wb'))

In [31]:
filename = 'RFC_news.sav'
pickle.dump(rfc, open(filename, 'wb'))

In [32]:
preprocessed_data = Train_X_Tfidf, Test_X_Tfidf, Train_Y, Test_Y
filename = 'preprocessed_data_news.sav'
pickle.dump(preprocessed_data, open(filename, 'wb'))

In [33]:
from google.colab import files
files.download('naive_news.sav')
files.download('svm_news.sav')
files.download('LogReg_news.sav')
files.download('RFC_news.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
files.download('preprocessed_data_news.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>